In [1]:
import os
import sys
sys.path.append('../../swdb_2019_tools')
import spikeutils
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.ndimage.filters import gaussian_filter1d
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys import ecephys_session
%matplotlib inline

# fix slow autocomplete
%config Completer.use_jedi = False

import platform
platstring = platform.platform()

if 'Darwin' in platstring:
    # OS X 
    data_root = "/Volumes/Brain2019/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn1' in platstring):
    # then on AWS
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2019/"

manifest_path = os.path.join(data_root, "dynamic-brain-workshop/visual_coding_neuropixels/2019/manifest.json")

In [2]:
cache = EcephysProjectCache.fixed(manifest=manifest_path)
sessions = cache.get_sessions()

areas = ['CA','DG']
area_sessions = [] # Adds all the sessions that have the desired areas (designated above)
for i in np.arange(len(sessions.structure_acronyms)):
    sessionid = sessions.structure_acronyms.index[i]
    if any(elem in sessions.structure_acronyms[sessionid] for elem in areas):
        area_sessions.append(sessionid)

session = cache.get_session_data(area_sessions[0])
stim_epochs = session.get_stimulus_epochs()
running_speed = session.running_speed

units = pd.read_csv('./CA_units_session_737581020_binned0to9000Secs1000binsize_GaussianSmoothed_sigma10.csv')
unit_id = units.iloc[:,0]
cell_type = units.iloc[:,1]
firingRates = units.iloc[:,2:].to_numpy()

/Users/josephzaki/anaconda3/envs/allenbrain/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [3]:
stim_epochs.head()

,start_time,stop_time,duration,stimulus_name,stimulus_block
0,28.998830,89.065570,60.066740,spontaneous,null
1,89.065570,1001.060513,911.994943,gabors,0
2,1001.060513,1290.051910,288.991397,spontaneous,null
3,1290.051910,1588.551172,298.499263,flashes,1
4,1588.551172,1590.302620,1.751447,spontaneous,null


In [4]:
epochlabels, epochuniques = pd.factorize(stim_epochs.stimulus_name)
startTime = 0
endtime = 9000
binsize = 1000
#assign labels
numlabels = len(np.unique(stim_epochs.stimulus_name))
numepochs = np.arange(numlabels)
labels = np.zeros((int(endtime*1000/binsize)))
for i in numepochs:
    epochstart = (stim_epochs.start_time[i]*1000)/binsize
    epochend = (stim_epochs.stop_time[i]*1000)/binsize
    stimname = stim_epochs.stimulus_name[i]
    labels[int(epochstart):int(epochend)] = np.where(epochuniques==stimname)[0][0]

In [8]:
# Choose all the sessions that have the given brain region
brainRegion = 'CA'
sessionsToUse = sessions[[np.isin(brainRegion, x) for x in sessions.structure_acronyms]]
sessionsToUse
# Get sample session to try binning and smoothing on
session = cache.get_session_data(sessionsToUse.index[0])
units = session.units
regionUnits = units[units.structure_acronym == brainRegion]
spike_times = session.spike_times
regionSpikes = {unit : spike_times[unit] for unit in regionUnits.index}
unitNames = np.asarray([n for n in regionSpikes.keys()])
toBin = [sp for sp in regionSpikes.values()]
startTime = 0
endTime = np.max([np.max(x) for x in spike_times.values()]) # Last spike across all cells in this session
binsize = 1000
binned = spikeutils.spiketimes_to_2D_rates(toBin, startime=startTime, stoptime=endTime, binsize=binsize)
stim_epochs = session.get_stimulus_epochs()
epochlabels, epochuniques = pd.factorize(stim_epochs.stimulus_name)
#assign labels
numlabels = len(np.unique(stim_epochs.stimulus_name))
numepochs = np.arange(numlabels)
labels = np.zeros((int(endTime*1000/binsize)))
for i in numepochs:
    epochstart = (stim_epochs.start_time[i]*1000)/binsize
    epochend = (stim_epochs.stop_time[i]*1000)/binsize
    stimname = stim_epochs.stimulus_name[i]
    labels[int(epochstart):int(epochend)] = np.where(epochuniques==stimname)[0][0]
epochmapping = {key:value for key, value in zip(np.arange(len(epochuniques)), epochuniques)}
labels = labels.astype(int)
labels_asNames = ['x']
for i in labels:
    labels_asNames.append(epochmapping[i])
labels_asNames = labels_asNames[1:]

/Users/josephzaki/anaconda3/envs/allenbrain/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [23]:
####SaM fixing  your shit

numepochs = len(stim_epochs.start_time)
labels = np.zeros((int(endTime*1000/binsize)))
for i in range(numepochs):
    epochstart = (stim_epochs.start_time[i]*1000)/binsize
    epochend = (stim_epochs.stop_time[i]*1000)/binsize
    stimname = stim_epochs.stimulus_name[i]
    labels[int(epochstart):int(epochend)] = np.where(epochuniques==stimname)[0][0]
epochmapping = {key:value for key, value in zip(np.arange(len(epochuniques)), epochuniques)}
labels = labels.astype(int)

In [25]:
np.unique(labels)

array([0, 1, 2, 3, 4, 5, 6, 7])

In [11]:
epochuniques

Index(['spontaneous', 'gabors', 'flashes', 'drifting_gratings',
       'natural_movie_three', 'natural_movie_one', 'static_gratings',
       'natural_scenes'],
      dtype='object')

In [ ]:
epochmapping = {key:value for key, value in zip(np.arange(len(epochuniques)), epochuniques)}
labels = labels.astype(int)
labels_asNames = ['x']
for i in labels:
    labels_asNames.append(epochmapping[i])
labels_asNames = labels_asNames[1:]

In [ ]:
description = "CA_stim_epochs_nums"
np.savetxt(('./' + description + '_session_' + str(area_sessions[0]) + "_binned" +
                      str(startTime) + "to" + str(endtime) + "Secs" + str(binsize) + "binsize" + '.csv'),labels,
           delimiter=',', fmt='%s')